# TerminatorModel End-to-End Demo

This notebook demonstrates KMR's TerminatorModel for advanced hierarchical feature processing with context. It includes:

- Data generation with input and context features
- Model creation, training, and evaluation
- Plotly visualizations
- Model serialization and loading validation

## 1. Setup and Imports


In [1]:
import os
import tempfile

import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import MeanAbsoluteError

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# KMR imports
from kerasfactory.models.TerminatorModel import TerminatorModel

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


✅ All imports successful!
TensorFlow version: 2.18.0
Keras version: 3.8.0


## 2. Generate Synthetic Data

We'll create a synthetic dataset with:
- Input features (primary features)
- Context features (additional contextual information)
- A continuous target that depends on both inputs


In [2]:
print("📦 Generating synthetic data...")

# Reproducibility
np.random.seed(42)

num_samples = 2000
input_dim = 16  # Primary features
context_dim = 8  # Context features

# Generate input features
X_input = np.random.randn(num_samples, input_dim).astype(np.float32)
X_input[:, 1] = 0.3 * X_input[:, 0] + 0.7 * X_input[:, 1]  # introduce correlation

# Generate context features
X_context = np.random.randn(num_samples, context_dim).astype(np.float32)
X_context[:, 0] = 0.5 * X_input[:, 0] + 0.3 * np.random.randn(num_samples).astype(np.float32)

# Create target that depends on both input and context
input_weights = np.linspace(1.0, 0.3, input_dim)
context_weights = np.linspace(0.8, 0.2, context_dim)

y_raw = (
    2.0 * np.sin(X_input[:, 0])
    + 0.5 * X_input[:, 1] ** 2
    - 1.2 * X_context[:, 0]
    + (X_input @ input_weights) * 0.5
    + (X_context @ context_weights) * 0.4
    + 0.3 * (X_input[:, 2] * X_context[:, 1])  # interaction term
    + 0.4 * np.random.randn(num_samples)
)

# Scale to [0, 1] range for sigmoid output (TerminatorModel uses sigmoid)
y_min, y_max = y_raw.min(), y_raw.max()
y = ((y_raw - y_min) / (y_max - y_min + 1e-8)).astype(np.float32)

print(f"Target range: [{y.min():.4f}, {y.max():.4f}] (scaled for sigmoid output)")

# Train/Val/Test split
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)

X_input_train = X_input[:train_size]
X_context_train = X_context[:train_size]
y_train = y[:train_size]

X_input_val = X_input[train_size:train_size + val_size]
X_context_val = X_context[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_input_test = X_input[train_size + val_size:]
X_context_test = X_context[train_size + val_size:]
y_test = y[train_size + val_size:]

print(f"✅ Data shapes:")
print(f"  Input features -> Train: {X_input_train.shape}, Val: {X_input_val.shape}, Test: {X_input_test.shape}")
print(f"  Context features -> Train: {X_context_train.shape}, Val: {X_context_val.shape}, Test: {X_context_test.shape}")
print(f"  Targets -> Train: {y_train.shape}, Val: {y_val.shape}, Test: {y_test.shape}")


📦 Generating synthetic data...
Target range: [0.0000, 1.0000] (scaled for sigmoid output)
✅ Data shapes:
  Input features -> Train: (1400, 16), Val: (300, 16), Test: (300, 16)
  Context features -> Train: (1400, 8), Val: (300, 8), Test: (300, 8)
  Targets -> Train: (1400,), Val: (300,), Test: (300,)


## 3. Build TerminatorModel

TerminatorModel stacks multiple SFNE blocks for hierarchical feature processing and uses context features to guide the processing.


In [3]:
# Create TerminatorModel
model = TerminatorModel(
    input_dim=input_dim,
    context_dim=context_dim,
    output_dim=1,
    hidden_dim=64,
    num_layers=2,
    num_blocks=3,
    slow_network_layers=3,
    slow_network_units=128,
    name='terminator_demo'
)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=MeanSquaredError(),
    metrics=[MeanAbsoluteError()],
)

print("✅ TerminatorModel created and compiled!")
print(f"Input dimension: {model.input_dim}")
print(f"Context dimension: {model.context_dim}")
print(f"Output dimension: {model.output_dim}")
print(f"Number of SFNE blocks: {model.num_blocks}")


2025-11-13 10:48:03.756 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 8, 'num_layers': 3, 'units': 128}
2025-11-13 10:48:03.758 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized HyperZZWOperator with parameters: {'name': 'hyper_zzw_operator', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 16, 'context_dim': 8}
2025-11-13 10:48:03.763 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 16, 'num_l

✅ TerminatorModel created and compiled!
Input dimension: 16
Context dimension: 8
Output dimension: 1
Number of SFNE blocks: 3


## 4. Train and Evaluate

TerminatorModel expects inputs as a list: `[input_features, context_features]`.


In [4]:
print("🚀 Starting training...")

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
]

history = model.fit(
    [X_input_train, X_context_train], y_train,
    validation_data=([X_input_val, X_context_val], y_val),
    epochs=30,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("✅ Training completed!")

# Evaluate
val_loss, val_mae = model.evaluate([X_input_val, X_context_val], y_val, verbose=0)
print(f"Validation - Loss: {val_loss:.4f}, MAE: {val_mae:.4f}")

test_loss, test_mae = model.evaluate([X_input_test, X_context_test], y_test, verbose=0)
print(f"Test - Loss: {test_loss:.4f}, MAE: {test_mae:.4f}")

# Predictions for later visualizations
y_pred_val = model.predict([X_input_val, X_context_val], verbose=0).squeeze()
y_pred_test = model.predict([X_input_test, X_context_test], verbose=0).squeeze()


🚀 Starting training...
Epoch 1/30


2025-11-13 10:48:03.848 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:03.864 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:03.871 | DEBUG    | kerasfactory.layers.SlowNetwork:build:106 - SlowNetwork built with input_dim=8, num_layers=3, units=128
2025-11-13 10:48:03.875 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:03.885 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:03.892 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:03.905 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:03.918 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 -

36/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0333 - mean_absolute_error: 0.1454

2025-11-13 10:48:07.776 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:07.778 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:07.778 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:07.781 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:07.783 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:07.786 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:07.788 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (None, 8)
2025-11-13 10:48:07.788 | DEBUG    | kerasfactory.models.TerminatorModel:call:211 - TerminatorModel hype

44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0326 - mean_absolute_error: 0.1438 - val_loss: 0.0225 - val_mean_absolute_error: 0.1188 - learning_rate: 0.0010
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0169 - mean_absolute_error: 0.1025 - val_loss: 0.0149 - val_mean_absolute_error: 0.0985 - learning_rate: 0.0010
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0115 - mean_absolute_error: 0.0857 - val_loss: 0.0109 - val_mean_absolute_error: 0.0830 - learning_rate: 0.0010
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0084 - mean_absolute_error: 0.0724 - val_loss: 0.0091 - val_mean_absolute_error: 0.0764 - learning_rate: 0.0010
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0061 - mean_absolute_error: 0.0623 - val_loss: 0.0074 - val_mean_absolute_error: 0.0686 - learning_rate: 0.0010
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0052 - mean_absolute_error: 0.0577 - val_loss: 0.0070 - val_mean_absolute_error: 0.0663 - 

2025-11-13 10:48:11.821 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (32, 16), context shape: (32, 8)
2025-11-13 10:48:11.823 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (32, 16)
2025-11-13 10:48:11.824 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (32, 8)
2025-11-13 10:48:11.826 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (32, 128)
2025-11-13 10:48:11.827 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (32, 128)
2025-11-13 10:48:11.829 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (32, 128)
2025-11-13 10:48:11.831 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (32, 8)
2025-11-13 10:48:11.831 | DEBUG    | kerasfactory.models.TerminatorModel:call:211 - TerminatorModel hyper_kernels shape:

## 5. Visualizations

We will plot training curves, predictions vs. ground truth, and residuals.


In [5]:
print("📊 Creating visualizations...")

# Loss curves
hist_loss = history.history.get("loss", [])
hist_val_loss = history.history.get("val_loss", [])

# Unscale predictions for better visualization (convert back from [0,1] range)
# Recalculate scaling parameters from the original data
y_test_raw = y_test * (y_raw.max() - y_raw.min()) + y_raw.min()
y_pred_test_raw = y_pred_test * (y_raw.max() - y_raw.min()) + y_raw.min()

# Residuals (test, unscaled)
residuals = (y_test_raw - y_pred_test_raw)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Training/Validation Loss",
        "Predictions vs Ground Truth (Test)",
        "Residuals Histogram (Test)",
        "MAE over Bins (Test)"
    ),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "histogram"}, {"type": "bar"}]]
)

# Plot 1: Loss curves
fig.add_scatter(y=hist_loss, mode="lines", name="loss", row=1, col=1)
fig.add_scatter(y=hist_val_loss, mode="lines", name="val_loss", row=1, col=1)

# Plot 2: Predictions vs truth (unscaled)
idx = np.random.choice(len(y_test_raw), size=min(400, len(y_test_raw)), replace=False)
fig.add_scatter(
    x=y_test_raw[idx], y=y_pred_test_raw[idx], mode="markers", name="pred vs true",
    row=1, col=2
)
# Reference line (unscaled)
min_v = float(min(y_test_raw.min(), y_pred_test_raw.min()))
max_v = float(max(y_test_raw.max(), y_pred_test_raw.max()))
fig.add_scatter(x=[min_v, max_v], y=[min_v, max_v], mode="lines", name="ideal", row=1, col=2)

# Plot 3: Residuals histogram
fig.add_histogram(x=residuals, nbinsx=40, name="residuals", row=2, col=1)

# Plot 4: MAE over bins
bins = np.linspace(min_v, max_v, 20)
bin_indices = np.digitize(y_test_raw, bins)
mae_per_bin = []
centers = []
for b in range(1, len(bins)):
    mask = bin_indices == b
    if np.any(mask):
        mae_per_bin.append(np.mean(np.abs(y_test_raw[mask] - y_pred_test_raw[mask])))
        centers.append((bins[b] + bins[b-1]) / 2)
    else:
        mae_per_bin.append(0.0)
        centers.append((bins[b] + bins[b-1]) / 2)

fig.add_bar(x=centers, y=mae_per_bin, name="MAE per bin", row=2, col=2)

fig.update_layout(height=800, title_text="TerminatorModel Regression Results", showlegend=True)
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_xaxes(title_text="True", row=1, col=2)
fig.update_yaxes(title_text="Predicted", row=1, col=2)
fig.update_xaxes(title_text="Residual", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_xaxes(title_text="Target (bin center)", row=2, col=2)
fig.update_yaxes(title_text="MAE", row=2, col=2)

fig.show()
print("✅ Visualizations created successfully!")


📊 Creating visualizations...


✅ Visualizations created successfully!


## 6. Model Serialization and Loading

We will save the TerminatorModel in Keras format and verify that a loaded model produces consistent predictions.


In [6]:
print("💾 Testing Keras format serialization...")

with tempfile.TemporaryDirectory() as temp_dir:
    keras_path = os.path.join(temp_dir, "terminator_demo.keras")

    # Save
    model.save(keras_path)
    print(f"✅ Model saved to: {keras_path}")

    # Load
    loaded_model = keras.models.load_model(keras_path)
    print("✅ Model loaded successfully!")

    # Compare predictions on a small slice
    sl = slice(0, 64)
    preds_orig = model.predict([X_input_test[sl], X_context_test[sl]], verbose=0)
    preds_loaded = loaded_model.predict([X_input_test[sl], X_context_test[sl]], verbose=0)

    # Report similarity
    diff = np.mean(np.abs(preds_orig - preds_loaded))
    print(f"🔍 Mean absolute difference between original and loaded predictions: {float(diff):.6f}")
    print("✅ Loaded model prediction check completed!")


2025-11-13 10:48:12.299 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_4', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 8, 'num_layers': 3, 'units': 128}
2025-11-13 10:48:12.299 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized HyperZZWOperator with parameters: {'name': 'hyper_zzw_operator_4', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 13842188672}, 'input_dim': 16, 'context_dim': 8}
2025-11-13 10:48:12.301 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_5', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_

💾 Testing Keras format serialization...
✅ Model saved to: /var/folders/v8/4l9cyywn1x970gdc1v67r5480000gn/T/tmphzwzixu8/terminator_demo.keras


2025-11-13 10:48:12.386 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 64)
2025-11-13 10:48:12.389 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:12.391 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:12.394 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:12.396 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (None, 16)
2025-11-13 10:48:12.397 | DEBUG    | kerasfactory.models.SFNEBlock:call:184 - SFNEBlock hyper_kernels shape: (None, 16)
2025-11-13 10:48:12.397 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:132 - HyperZZWOperator input_tensor shape: (None, 64)
2025-11-13 10:48:12.397 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:133 - HyperZZWOperator context_tensor shape: (None, 16)
2

✅ Model loaded successfully!
🔍 Mean absolute difference between original and loaded predictions: 0.000000
✅ Loaded model prediction check completed!


## 7. Summary

- Trained `TerminatorModel` on synthetic regression data with input and context features.
- Achieved regression metrics (reported above) on validation and test sets.
- Visualized loss curves, predictions vs. truth, and residuals.
- Saved and loaded model in Keras format; verified prediction consistency.

This notebook mirrors the style of the other KMR demos and showcases TerminatorModel's hierarchical feature processing with context-aware capabilities.


## 8. Alternative: Binary Classification Training (Better Fit)

The current `TerminatorModel` uses a sigmoid output, which is naturally suited for binary classification. Below we derive a binary target from the same features and train the model with a classification-appropriate loss and metrics.


In [7]:
import keras
from keras import layers
from kerasfactory.utils import KerasFactoryPlotter

print("🔧 Preparing binary classification targets and model...")
# Build probabilities from input + context for a non-trivial decision boundary
# 
w_inp = np.linspace(0.7, 0.2, input_dim)
w_ctx = np.linspace(0.5, 0.1, context_dim)
logits = (1.5 * np.sin(X_input[:, 0])+ 0.8 * (X_input[:, 1] ** 2) + 1.1 * X_context[:, 0] + 0.5 * (X_input @ w_inp)+ 0.4 * (X_context @ w_ctx)  + 0.35 * (X_input[:, 2] * X_context[:, 1])    + 0.2 * (X_input[:, 3] * X_context[:, 2]))
probs = 1 / (1 + np.exp(-logits))

# Ensure balanced classes
y_cls = (probs > np.median(probs)).astype(np.float32)

print(f"Class distribution: {np.bincount(y_cls.astype(int))}")
y_train_cls = y_cls[:train_size]
y_val_cls = y_cls[train_size:train_size + val_size]
y_test_cls = y_cls[train_size + val_size:]

# Fresh model for classification
model_cls = TerminatorModel(
    input_dim=input_dim,
    context_dim=context_dim,
    output_dim=1,
    hidden_dim=64,
    num_layers=2,
    num_blocks=3,
    slow_network_layers=3,
    slow_network_units=128,
    name='terminator_demo_cls',
)
    
model_cls.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'), 
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
        ],
)
print("🚀 Training classification model...")
cls_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
    )]

history_cls = model_cls.fit(
    [X_input_train, X_context_train],
    y_train_cls,
    validation_data=([X_input_val, X_context_val], y_val_cls),
    epochs=30,
    batch_size=64,
    callbacks=cls_callbacks,
    verbose=1,
)
print("✅ Classification training completed!")

# Evaluate and predict

loss_t, acc_t, prec_t, rec_t, auc_t = model_cls.evaluate(
    [X_input_test, X_context_test],
    y_test_cls,
    verbose=0,
)

print(f"Test -> loss {loss_t:.4f}, acc {acc_t:.4f}, prec {prec_t:.4f}, recall {rec_t:.4f}, auc {auc_t:.4f}")
y_pred_proba_cls = model_cls.predict([X_input_test, X_context_test], verbose=0).flatten()
y_pred_cls = (y_pred_proba_cls > 0.5).astype(np.int32)

2025-11-13 10:48:12.940 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_8', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 8, 'num_layers': 3, 'units': 128}
2025-11-13 10:48:12.976 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized HyperZZWOperator with parameters: {'name': 'hyper_zzw_operator_8', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 16, 'context_dim': 8}
2025-11-13 10:48:13.003 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_9', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 16, 'n

🔧 Preparing binary classification targets and model...
Class distribution: [1000 1000]
🚀 Training classification model...
Epoch 1/30


2025-11-13 10:48:13.078 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:13.086 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:13.089 | DEBUG    | kerasfactory.layers.SlowNetwork:build:106 - SlowNetwork built with input_dim=8, num_layers=3, units=128
2025-11-13 10:48:13.089 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:13.099 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:13.107 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:13.121 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:13.154 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 -

12/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4827 - auc: 0.5057 - loss: 0.7243 - precision: 0.5104 - recall: 0.3255 

2025-11-13 10:48:17.183 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:17.185 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:17.186 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:17.188 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:17.191 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:17.193 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:17.194 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (None, 8)
2025-11-13 10:48:17.195 | DEBUG    | kerasfactory.models.TerminatorModel:call:211 - TerminatorModel hype

22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.4905 - auc: 0.5101 - loss: 0.7207 - precision: 0.5078 - recall: 0.3507 - val_accuracy: 0.5600 - val_auc: 0.6122 - val_loss: 0.6793 - val_precision: 0.5394 - val_recall: 0.6138 - learning_rate: 0.0010
Epoch 2/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6703 - auc: 0.7230 - loss: 0.6207 - precision: 0.6612 - recall: 0.7386 - val_accuracy: 0.8000 - val_auc: 0.8819 - val_loss: 0.4460 - val_precision: 0.8400 - val_recall: 0.7241 - learning_rate: 0.0010
Epoch 3/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8090 - auc: 0.8899 - loss: 0.4262 - precision: 0.8504 - recall: 0.7732 - val_accuracy: 0.8567 - val_auc: 0.9300 - val_loss: 0.3369 - val_precision: 0.8355 - val_recall: 0.8759 - learning_rate: 0.0010
Epoch 4/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8517 - auc: 0.9295 - loss: 0.3449 - precision: 0.8474 - recall: 0.8538 - val_accuracy: 0.8500 - val_auc: 0.9384 - val_loss: 0.3094 - val_precision: 0.8333

2025-11-13 10:48:20.396 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (32, 16), context shape: (32, 8)
2025-11-13 10:48:20.399 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (32, 16)
2025-11-13 10:48:20.399 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (32, 8)
2025-11-13 10:48:20.401 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (32, 128)
2025-11-13 10:48:20.403 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (32, 128)
2025-11-13 10:48:20.404 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (32, 128)
2025-11-13 10:48:20.406 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (32, 8)
2025-11-13 10:48:20.406 | DEBUG    | kerasfactory.models.TerminatorModel:call:211 - TerminatorModel hyper_kernels shape:

Test -> loss 0.2643, acc 0.8933, prec 0.9104, recall 0.8592, auc 0.9599


2025-11-13 10:48:20.423 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:132 - HyperZZWOperator input_tensor shape: (32, 64)
2025-11-13 10:48:20.424 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:133 - HyperZZWOperator context_tensor shape: (32, 16)
2025-11-13 10:48:20.425 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:153 - HyperZZWOperator output shape: (32, 64)
2025-11-13 10:48:20.425 | DEBUG    | kerasfactory.models.SFNEBlock:call:188 - SFNEBlock context_weights shape: (32, 64)
2025-11-13 10:48:20.426 | DEBUG    | kerasfactory.models.SFNEBlock:call:192 - SFNEBlock context_weights_expanded shape: (32, 64, 1)
2025-11-13 10:48:20.430 | DEBUG    | kerasfactory.models.SFNEBlock:call:198 - SFNEBlock global_output shape: (32, 64, 16)
2025-11-13 10:48:20.434 | DEBUG    | kerasfactory.models.SFNEBlock:call:202 - SFNEBlock local_output shape: (32, 64, 16)
2025-11-13 10:48:20.435 | DEBUG    | kerasfactory.models.SFNEBlock:call:206 - SFNEBlock combined_output shape: (32, 

In [8]:
# Classification visualizations
print("📊 Creating classification visualizations...")

fig_cls = KerasFactoryPlotter.create_comprehensive_plot(
    'classification',
    y_true=y_test_cls,
    y_pred=y_pred_cls,
    y_scores=y_pred_proba_cls,
    title="TerminatorModel Classification Results"
)
fig_cls.show()
print("✅ Classification visualizations created!")


📊 Creating classification visualizations...


✅ Classification visualizations created!


## 9. Scenario 2: With Preprocessing Model

We now apply a preprocessing model that consumes `input` and `context`, projects them to a common space, combines, and outputs a processed representation that `TerminatorModel` then uses. We'll feed inputs as a dictionary to leverage universal input handling.


In [9]:
# Build a preprocessing model that works with TerminatorModel's standardized input format
# TerminatorModel converts list inputs to {'input_0': input, 'input_1': context}
from keras import Input, Model

print("🧩 Building preprocessing model...")

# Input layers - using the standardized format that TerminatorModel provides
inp_0 = Input(shape=(input_dim,), name='input_0')  # This will be the input features
inp_1 = Input(shape=(context_dim,), name='input_1')  # This will be the context features

# Process each input
h1 = layers.Dense(64, activation='relu', name='preproc_dense1')(inp_0)
h1 = layers.Dropout(0.1)(h1)
h2 = layers.Dense(32, activation='relu', name='preproc_dense2')(inp_1)
h2 = layers.Dropout(0.1)(h2)

# Combine and process
combined = layers.Concatenate(name='preproc_concat')([h1, h2])
combined = layers.Dense(64, activation='relu', name='preproc_dense3')(combined)
combined = layers.Dropout(0.1)(combined)
processed = layers.Dense(input_dim, activation='linear', name='processed')(combined)

# Create preprocessing model with the standardized input format
preproc_model = Model(
    inputs={'input_0': inp_0, 'input_1': inp_1}, 
    outputs=processed, 
    name='terminator_preproc'
)

# Test the preprocessing model with the standardized format
print("Testing preprocessing model...")
test_inp = np.random.randn(1, input_dim).astype(np.float32)
test_ctx = np.random.randn(1, context_dim).astype(np.float32)
test_out = preproc_model({'input_0': test_inp, 'input_1': test_ctx})
print(f"Preprocessing model output shape: {test_out.shape}, expected: (1, {input_dim})")
assert test_out.shape[1] == input_dim, f"Preprocessing output shape mismatch!"

# Model with preprocessing
model_cls_preproc = TerminatorModel(
    input_dim=input_dim,
    context_dim=context_dim,
    output_dim=1,
    hidden_dim=64,
    num_layers=2,
    num_blocks=3,
    slow_network_layers=3,
    slow_network_units=128,
    preprocessing_model=preproc_model,
    name='terminator_demo_cls_preproc'
)

model_cls_preproc.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print("🚀 Training classification model with preprocessing...")

# Test the model with preprocessing first
print("Testing model with preprocessing...")
# Use list format since that's what TerminatorModel expects
test_output = model_cls_preproc([X_input_train[:1], X_context_train[:1]])
print(f"Model with preprocessing test output shape: {test_output.shape}")

# Train with list format (which works with the preprocessing model)
print("Training with list inputs...")
history_cls_preproc = model_cls_preproc.fit(
    [X_input_train, X_context_train], y_train_cls,
    validation_data=([X_input_val, X_context_val], y_val_cls),
    epochs=30,
    batch_size=64,
    callbacks=cls_callbacks,
    verbose=1
)

print("✅ Training with preprocessing completed!")

# Evaluate and visualize (use list format consistently)
loss_p, acc_p, prec_p, rec_p, auc_p = model_cls_preproc.evaluate(
    [X_input_test, X_context_test], y_test_cls, verbose=0
)
y_pred_proba_cls_p = model_cls_preproc.predict(
    [X_input_test, X_context_test], verbose=0
).flatten()

print(f"Test (preproc) -> loss {loss_p:.4f}, acc {acc_p:.4f}, prec {prec_p:.4f}, recall {rec_p:.4f}, auc {auc_p:.4f}")

y_pred_cls_p = (y_pred_proba_cls_p > 0.5).astype(np.int32)

fig_cls_p = KerasFactoryPlotter.create_comprehensive_plot(
    'classification',
    y_true=y_test_cls,
    y_pred=y_pred_cls_p,
    y_scores=y_pred_proba_cls_p,
    title="TerminatorModel Classification Results (With Preprocessing)"
)
fig_cls_p.show()
print("✅ Classification visualizations with preprocessing created!")


2025-11-13 10:48:20.877 | DEBUG    | kerasfactory.models._base:_setup_preprocessing_model:348 - Setting up preprocessing model integration
2025-11-13 10:48:20.878 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_12', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 8, 'num_layers': 3, 'units': 128}
2025-11-13 10:48:20.878 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized HyperZZWOperator with parameters: {'name': 'hyper_zzw_operator_12', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 16, 'context_dim': 8}
2025-11-13 10:48:20.879 | DEBUG    | kerasfactory.layers._base_layer:_log_initialization:73 - Initialized SlowNetwork with parameters: {'name': 'slow_network_13', 'trainable'

🧩 Building preprocessing model...
Testing preprocessing model...
Preprocessing model output shape: (1, 16), expected: (1, 16)
🚀 Training classification model with preprocessing...
Testing model with preprocessing...


2025-11-13 10:48:20.945 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (1, 16)
2025-11-13 10:48:20.945 | DEBUG    | kerasfactory.models.SFNEBlock:call:184 - SFNEBlock hyper_kernels shape: (1, 16)
2025-11-13 10:48:20.946 | DEBUG    | kerasfactory.layers.HyperZZWOperator:build:112 - HyperZZWOperator built with input_dim=64, context_dim=16
2025-11-13 10:48:20.946 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:132 - HyperZZWOperator input_tensor shape: (1, 64)
2025-11-13 10:48:20.947 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:133 - HyperZZWOperator context_tensor shape: (1, 16)
2025-11-13 10:48:20.950 | DEBUG    | kerasfactory.layers.HyperZZWOperator:call:153 - HyperZZWOperator output shape: (1, 64)
2025-11-13 10:48:20.950 | DEBUG    | kerasfactory.models.SFNEBlock:call:188 - SFNEBlock context_weights shape: (1, 64)
2025-11-13 10:48:20.951 | DEBUG    | kerasfactory.models.SFNEBlock:call:192 - SFNEBlock context_weights_expanded shape: (

Model with preprocessing test output shape: (1, 1)
Training with list inputs...
Epoch 1/30


/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kerasfactory-5WgtroCZ-py3.12/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: {'input_0': 'input_0', 'input_1': 'input_1'}
Received: inputs=['Tensor(shape=(None, 16))', 'Tensor(shape=(None, 8))']

2025-11-13 10:48:21.293 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:21.295 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:21.295 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:21.297 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:21.300 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape

...

2025-11-13 10:48:25.155 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (None, 16), context shape: (None, 8)
2025-11-13 10:48:25.157 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (None, 16)
2025-11-13 10:48:25.158 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (None, 8)
2025-11-13 10:48:25.160 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (None, 128)
2025-11-13 10:48:25.162 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (None, 128)
2025-11-13 10:48:25.164 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 2 output shape: (None, 128)
2025-11-13 10:48:25.166 | DEBUG    | kerasfactory.layers.SlowNetwork:call:129 - SlowNetwork output shape: (None, 8)
2025-11-13 10:48:25.166 | DEBUG    | kerasfactory.models.TerminatorModel:call:211 - TerminatorModel hype

22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.5423 - auc: 0.5557 - loss: 0.7249 - precision: 0.5391 - recall: 0.6468 - val_accuracy: 0.5967 - val_auc: 0.6207 - val_loss: 0.6669 - val_precision: 0.5759 - val_recall: 0.6276 - learning_rate: 0.0010
Epoch 2/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6665 - auc: 0.7361 - loss: 0.5924 - precision: 0.6950 - recall: 0.6065 - val_accuracy: 0.8733 - val_auc: 0.9514 - val_loss: 0.2986 - val_precision: 0.8408 - val_recall: 0.9103 - learning_rate: 0.0010
Epoch 3/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8745 - auc: 0.9504 - loss: 0.2953 - precision: 0.8438 - recall: 0.9236 - val_accuracy: 0.8800 - val_auc: 0.9577 - val_loss: 0.2853 - val_precision: 0.8344 - val_recall: 0.9379 - learning_rate: 0.0010
Epoch 4/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9066 - auc: 0.9665 - loss: 0.2398 - precision: 0.9007 - recall: 0.9209 - val_accuracy: 0.8967 - val_auc: 0.9638 - val_loss: 0.2479 - val_precision: 0.8750

/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kerasfactory-5WgtroCZ-py3.12/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: {'input_0': 'input_0', 'input_1': 'input_1'}
Received: inputs=['Tensor(shape=(32, 16))', 'Tensor(shape=(32, 8))']

2025-11-13 10:48:27.836 | DEBUG    | kerasfactory.models.TerminatorModel:call:201 - TerminatorModel input shape: (32, 16), context shape: (32, 8)
2025-11-13 10:48:27.838 | DEBUG    | kerasfactory.models.TerminatorModel:call:207 - TerminatorModel input_layer output shape: (32, 16)
2025-11-13 10:48:27.838 | DEBUG    | kerasfactory.layers.SlowNetwork:call:122 - SlowNetwork input shape: (32, 8)
2025-11-13 10:48:27.841 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 0 output shape: (32, 128)
2025-11-13 10:48:27.843 | DEBUG    | kerasfactory.layers.SlowNetwork:call:126 - SlowNetwork layer 1 output shape: (32, 128)
20

Test (preproc) -> loss 0.1827, acc 0.9267, prec 0.8947, recall 0.9577, auc 0.9780


✅ Classification visualizations with preprocessing created!
